In [1]:
# Analysis

import pandas as pd
import numpy as np

# Read cleaned csv
df = pd.read_csv("/Users/thomassimmons/c/p/data/df_clean.csv")

In [2]:
# Detect columns
num = df.select_dtypes(include=np.number).columns.tolist()
cat = df.select_dtypes(include=["object", "category"]).columns.tolist()

In [3]:
# Keep small-cardinality categoricals only (<= 20 unique)
cat_small = [c for c in cat if df[c].nunique(dropna=True) <= 20]

print("Numeric cols:", num[:8])
print("Categorical (small) cols:", cat_small[:5])

Numeric cols: ['NDC', 'Labeler.Code', 'Product.Code', 'Package.Size', 'Year', 'Quarter', 'Units.Reimbursed', 'Number.of.Prescriptions']
Categorical (small) cols: ['Utilization.Type']


In [4]:
# Overall numeric summary (mean/sum/count non-null)
overall = pd.DataFrame({
    "metric": ["mean", "sum", "count_nonnull"]
})
for c in num[:5]:
    overall[f"{c}"] = [df[c].mean(), df[c].sum(), df[c].notna().sum()]
print("\nOverall numeric summary (first 5 numeric cols):")
print(overall)


Overall numeric summary (first 5 numeric cols):
          metric           NDC  Labeler.Code  Product.Code  Package.Size  \
0           mean  4.101116e+10  4.101103e+04  1.245119e+03  2.134281e+01   
1            sum  1.066187e+17  1.066184e+11  3.236996e+09  5.548594e+07   
2  count_nonnull  2.599748e+06  2.599748e+06  2.599748e+06  2.599748e+06   

           Year  
0  2.024000e+03  
1  5.261890e+09  
2  2.599748e+06  


In [5]:
# Group-by: first small categorical vs first few numeric
if cat_small and num:
    gcol = cat_small[0]
    gb = df.groupby(gcol)[num[:5]].agg(["mean", "sum", "count"])
    gb.columns = ["_".join(col).strip() for col in gb.columns.to_flat_index()]
    gb = gb.sort_values(by="{}_count".format(num[0]), ascending=False)
    print(f"\nGroup-by '{gcol}' (first 5 numeric cols):")
    print(gb.head(10))
else:
    print("\nNo suitable categorical or numeric columns for group-by")


Group-by 'Utilization.Type' (first 5 numeric cols):
                      NDC_mean            NDC_sum  NDC_count  \
Utilization.Type                                               
MCOU              4.073465e+10  57911266156661103    1421671   
FFSU              4.134485e+10  48707412625431236    1178077   

                  Labeler.Code_mean  Labeler.Code_sum  Labeler.Code_count  \
Utilization.Type                                                            
MCOU                   40734.518533       57911083698             1421671   
FFSU                   41344.726473       48707271329             1178077   

                  Product.Code_mean  Product.Code_sum  Product.Code_count  \
Utilization.Type                                                            
MCOU                    1283.195443        1824281748             1421671   
FFSU                    1199.169762        1412714316             1178077   

                  Package.Size_mean  Package.Size_sum  Package.Size_coun